# Generador de preguntas de Moodle

In [8]:
# Paquetes
import itertools
import os
import random

import numpy as np
from scipy.stats import norm

## Funciones adicionales


In [9]:
def DistrNormal(media, sigma, unaD):
    """
    Genera una expresión para calcular una probabilidad en una distribución normal
    con un parámetro (P(X <= a) o P(X >= a)) o con dos parámetros (P(a <= X <= b)).
    Devuelve el enunciado del problema junto con el resultado calculado.

    Parámetros:
    media: Media de la distribución normal.
    sigma: Desviación estándar de la distribución normal.
    unaD: Booleano. Si es True, genera una desigualdad con un solo parámetro.
          Si es False, genera una desigualdad con dos parámetros.

    Retorna:
    Enunciado del problema y el resultado calculado.
    """

    # Cálculo y redondeo de la varianza
    varianza = round(sigma ** 2, 2)

    # Definición de la distribución de X
    distribucion = f'Sea \\( X \\sim \\Norm({media}, {varianza}) \\).'

    # Elección de dos valores aleatorios dentro del rango [-3*sigma, 3*sigma]
    k1 = (6 * sigma) * random.random()
    k2 = (6 * sigma) * random.random()

    # Cálculo de los límites a y b
    a = round(media - 3 * sigma + min(k1, k2), 2)
    b = round(media - 3 * sigma + max(k1, k2), 2)

    # Selección aleatoria de la desigualdad
    D = random.random()

    # Enunciado y cálculo para una desigualdad
    if unaD:
        if D < 0.5:
            desigualdad = r' \leq '
            resultado = norm.cdf(x=a, loc=media, scale=sigma)
        else:
            desigualdad = r' \geq '
            resultado = 1 - norm.cdf(x=a, loc=media, scale=sigma)

        enunciado = f'{distribucion} Calcule \\( P(X{desigualdad}{a}) \\).'

    # Enunciado y cálculo para dos desigualdades
    else:
        desigualdad = r' \leq X \leq '
        resultado = norm.cdf(x=b, loc=media, scale=sigma) - norm.cdf(x=a, loc=media, scale=sigma)

        enunciado = f'{distribucion} Calcule \\( P({a}{desigualdad}{b}) \\).'

    return enunciado, round(resultado, 4)

## Parámetros y enunciado

In [10]:
# Nombre del cuestionario
name = "Normal"

# Parámetros
par = ["m", "u", "unaD"]
# Diccionario de parámetros
par_dict = {
    "m": range(3, 7),
    "u": [1, 2, 3],
    "unaD": [True, False],
}

# Enunciado
enunciado = r'''
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Normal - [[id]]}
    % - Enunciado
    [[enunciado_generado]]
    \item[] [[ res ]]
\end{numerical}
'''

# Cantidad de preguntas a generar
n_preguntas = 10

# Código adicional a compilar antes de la generación de cada pregunta (opcional)
funciones = [
    "enunciado_generado, res = DistrNormal(m, u, unaD)",
]
# Plantilla personalizada (opcional, por defento Plantilla.tex)
plantilla = "PlantillaAleph.tex"
# Carpeta de salida (opcional, por defecto output/)
# carpeta = "output/"

## Generar preguntas

In [11]:
# Total preguntas
_total = n_preguntas
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Semilla
random.seed(18)

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))
if len(_par_comb) < _total:
    _total = len(_par_comb)
else:
    _par_comb = random.sample(_par_comb, _total)

# Genero el examen
_quiz = ''
id = 1
if not 'funciones' in locals():
    funciones = []
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        for _funcion in funciones:
            exec(_funcion)
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)
    if id > _total:
        break

# Mensaje
print(_quiz)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Normal - 1}
    % - Enunciado
    Sea \( X \sim \Norm(3, 9) \). Calcule \( P(2.25 \leq X \leq 2.64) \).
    \item[] 0.0509
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Normal - 2}
    % - Enunciado
    Sea \( X \sim \Norm(3, 4) \). Calcule \( P(0.05 \leq X \leq 5.3) \).
    \item[] 0.8048
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Normal - 3}
    % - Enunciado
    Sea \( X \sim \Norm(6, 4) \). Calcule \( P(8.1 \leq X \leq 9.45) \).
    \item[] 0.1046
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Normal - 4}
    % - Enunciado
    Sea \( X \sim \Norm(5, 4) \). Calcule \( P(X \leq 1.81) \).
    \item[] 0.0554
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

## Generar .tex, .pdf y .xml

In [12]:
# Total preguntas
_total = n_preguntas
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Semilla
random.seed(18)

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))
if len(_par_comb) < _total:
    _total = len(_par_comb)
else:
    _par_comb = random.sample(_par_comb, _total)

# Genero el examen
_quiz = ''
id = 1
if not 'funciones' in locals():
    funciones = []
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        for _funcion in funciones:
            exec(_funcion)
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)
    if id > _total:
        break

_parametros_latex = ''
for _parametro in par:
    _parametros_latex += '\t' + fr'\item ${_parametro} \in ' + '\\{' + ', '.join(map(str, par_dict[_parametro])) + '\\}$' + '\n'

# Leo el archivo plantilla
# Si la variable plantilla no está definida, la defino como 'Plantilla.tex'
if not 'plantilla' in locals():
    plantilla = 'Plantilla.tex'
_plantilla = open(plantilla, 'r', encoding='utf-8')
_t_plantilla = _plantilla.read()
_plantilla.close()
# Reemplazo
_t_plantilla = _t_plantilla.replace('{{QUIZ}}', _quiz)
_t_plantilla = _t_plantilla.replace('{{Cuestionario}}', name)
_t_plantilla = _t_plantilla.replace('{{Parámetros}}', _parametros_latex)
_t_plantilla = _t_plantilla.replace('{{Número de preguntas}}', str(id-1))

# Escribo el archivo
# Si la variable carpeta no está definida, la defino como 'test'
if not 'carpeta' in locals():
    carpeta = 'output'
# Si no existe la carpeta, la creo
if not os.path.exists(carpeta):
    os.makedirs(carpeta)
_examen = open(f'{carpeta}/{name}.tex', 'w', encoding='utf-8')
_examen.write(_t_plantilla)
_examen.close()

# Compilo el examen con xeLaTeX
os.system(f'latexmk -xelatex "{name}.tex" -output-directory="./{carpeta}"')

# Elimino todos los archivos llamados examen, salvo el pdf y el xml
_archivos = os.listdir(f'./{carpeta}')
for _archivo in _archivos:
    if name in _archivo:
        if '.pdf' not in _archivo and '.xml' not in _archivo and '.tex' not in _archivo:
            try:
                os.remove(f"{carpeta}/"+_archivo)
            except:
                pass
# Elimino los archivos fls
for _archivo in _archivos:
    if '.fls' in _archivo:
        try:
            os.remove(_archivo)
        except:
            pass

# Mensaje
print(f'Examen generado con éxito, se generaron {id-1} preguntas.')

Examen generado con éxito, se generaron 10 preguntas.
